In [ ]:
import numpy as np
import pandas as pd

In [ ]:
#This notebook is created to run value analysis with your own model on binary datasets
#Use this code to tune the empirical threshold on test set while running the value analysis
#specify the required info here and run the notebook to receive value analysis result for your model
modelName = 'name_of_your_model'
resPath = 'define_the_path_for_results'
data_folder = 'define_the_path_where_you_keep_the_confidence_values_of_your_model_and_the_datasets'
confidencesToTest = 'name_of_the_numpy_array_for_the_confidences_on_test_set.npy'
dataToTest ='name_of_test_set.csv'
ground_truth_column = 'specify_the_column_for_ground_truth_in_your_csv_files'
txt = 'specify_the_column_for_text_in_your_csv_files'
datasetName = 'name_of_your_dataset'

In [ ]:
#result file
logfile_name = "{}_{}_optTest".format(datasetName)

#cost-based parameters
Vr = 0.0
Vc = 1.0

Vw_list_fn = list(np.arange(0, -10.1, -1))
Vw_list_fp = list(np.arange(0, -10.1, -1))

confT_list = list(np.arange(0, 1.01, 0.01))

In [ ]:
logits_test = np.load(data_folder  + confidencesToTest)
y_test_df = pd.read_csv(data_folder + dataToTest)
y_test = y_test_df[ground_truth_column].values

In [ ]:
def cost_based_threshold(k):
    t = (k)/(k+1)
    return t

def calculate_value(y_hat_proba, y, t_fp, V_fp, t_fn, V_fn, Vc, Vr):
    prob_positive = y_hat_proba[:,1]
    prob_negative = y_hat_proba[:,0]

    y_pred_pos = np.full(prob_positive.shape[0],-1) 
    y_pred_neg = np.full(prob_negative.shape[0],-1) 

    y_pred_pos[prob_positive > t_fp] = 1
    y_pred_neg[prob_negative > t_fn] = 0

    max_prob_indices = list(np.argmax(y_hat_proba, axis=1))

    y_pred = np.array([y_pred_neg[i] if max_prob_indices[i] == 0 else y_pred_pos[i] for i in range(len(max_prob_indices))])

    # now lets compute the actual value of each prediction
    value_vector = np.full(y_pred.shape[0], Vc)

    #loss due to false positives and false negatives
    false_positives_idx = (y_pred == 1) & ( y == 0)
    false_negatives_idx = (y_pred == 0) & ( y == 1)

    value_vector[false_positives_idx] = V_fp
    value_vector[false_negatives_idx] = V_fn

    #loss due to asking humans
    value_vector[y_pred == -1] = Vr

    value = np.sum(value_vector) / len(y)

    numOfRejectedSamples = np.count_nonzero(y_pred == -1)
    numOfWrongPredictions = np.count_nonzero((y_pred != y) & (y_pred != -1))
    return value, numOfRejectedSamples, numOfWrongPredictions

def find_optimum_confidence_threshold(y_hat_proba, y, t_list, Vw_fp, Vw_fn, Vc, Vr):

    cost_list = {}

    for t_fp in t_list:
        for t_fn in t_list:
            # here we define K = fn_c_norm, change it based on task. 
            value = calculate_value(y_hat_proba, y, t_fp, Vw_fp, t_fn, Vw_fn, Vc, Vr)
            cost_list["{}_{}".format(t_fp,t_fn)] = value
    # find t values with maximum value
    maxValue = max(cost_list.values())
    optTList = [[float(k.split('_')[0]),float(k.split('_')[1])] for k, v in cost_list.items() if v == maxValue]

    return optTList[0], cost_list

#cost based calibration analysis
def cost_based_analysis(y_hat_proba_test, y_test, res_path, logfile_name, Vr, Vc, Vw_list_fp, Vw_list_fn, confT_list):

    # create log file
    rc_path = res_path + logfile_name + "_costBased_test.csv"
    with open(rc_path, 'w') as f:
        c = 'Vr, Vc, Vw_fp, Vw_fn, k_fp, k_fn, t_fp, t_fn, value, rejected, wrong, t_optimal_fp, t_optimal_fn, value_optimal, rejected_opt, wrong_opt'
        f.write(c + '\n')

    for Vw_fp in Vw_list_fp:
        for Vw_fn in Vw_list_fn:
            k_fp = (-1)*(Vw_fp / Vc)
            k_fn = (-1)*(Vw_fn / Vc)
            t_fp = cost_based_threshold(k_fp)
            t_fn = cost_based_threshold(k_fn)
 
            value_test, rej_test, wrong_test = calculate_value(y_hat_proba_test, y_test, t_fp, Vw_fp, t_fn, Vw_fn, Vc, Vr)

            t_optimal, cost_list = find_optimum_confidence_threshold(y_hat_proba_test, y_test, confT_list, Vw_fp, Vw_fn, Vc, Vr)

            value_test_opt, rej_test_opt, wrong_test_opt = calculate_value(y_hat_proba_test, y_test, t_optimal[0], Vw_fp, t_optimal[1], Vw_fn, Vc, Vr)

            with open(rc_path, 'a') as f:
                res_i = '{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}\n'.format(Vr, Vc, Vw_fp, Vw_fn, k_fp, k_fn, t_fp, t_fn, value_test, rej_test, wrong_test, t_optimal[0], t_optimal[1], value_test_opt, rej_test_opt, wrong_test_opt)
                f.write(res_i)

In [ ]:
cost_based_analysis(logits_test, y_test, resPath, logfile_name, Vr, Vc, Vw_list_fp, Vw_list_fn, confT_list)